In [2]:
from transformers import AutoModel, AutoTokenizer
from transformers import GenerationConfig
import torch
from mtkresearch.llm.prompt import MRPromptV3

model_id = 'MediaTek-Research/Llama-Breeze2-3B-Instruct-v0_1'
model = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map='auto',
    img_context_token_id=128212
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)

generation_config = GenerationConfig(
  max_new_tokens=2048,
  do_sample=True,
  temperature=0.01,
  top_p=0.01,
  repetition_penalty=1.1,
  eos_token_id=128009
)

prompt_engine = MRPromptV3()

sys_prompt = 'You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.'

def _inference(tokenizer, model, generation_config, prompt, pixel_values=None):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    if pixel_values is None:
        output_tensors = model.generate(**inputs, generation_config=generation_config)
    else:
        output_tensors = model.generate(**inputs, generation_config=generation_config, pixel_values=pixel_values.to(model.device, dtype=model.dtype))
    output_str = tokenizer.decode(output_tensors[0])
    return output_str

c:\Users\user\Desktop\0801\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\Desktop\0801\.venv\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FlashAttention2 is not installed.


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.88s/it]


In [21]:
import os
import json
import torch
import sys
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from mtkresearch.llm.prompt import MRPromptV3

# 模型與分詞器設定
model_id = 'MediaTek-Research/Llama-Breeze2-3B-Instruct-v0_1'

# 針對 Windows 系統移除信號超時檢查，以避免 AttributeError
if sys.platform == "win32":
    import signal
    def no_op_handler(signum, frame):
        pass
    signal.signal = no_op_handler
    print("偵測到 Windows 環境，已調整信號處理以順利載入模型。")
    print("-" * 50)

# 載入模型
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        device_map='auto',
        img_context_token_id=128212
    ).eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)
    print("✅ 模型與分詞器載入成功！")
except Exception as e:
    print(f"❌ 載入模型時發生錯誤：{e}")
    sys.exit(1)

# 生成設定
generation_config = GenerationConfig(
    max_new_tokens=2048,
    do_sample=True,
    temperature=0.01,
    top_p=0.01,
    repetition_penalty=1.1,
    eos_token_id=128009
)

# 提示詞引擎與系統提示
prompt_engine = MRPromptV3()
sys_prompt = 'You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.'

def _inference(tokenizer, model, generation_config, prompt, pixel_values=None):
    """
    執行模型推論的函式
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    if pixel_values is None:
        # 修正：直接傳遞 pixel_values=None 給 generate，以滿足多模態模型的內部檢查
        output_tensors = model.generate(**inputs, generation_config=generation_config, pixel_values=None)
    else:
        output_tensors = model.generate(**inputs, generation_config=generation_config, pixel_values=pixel_values.to(model.device, dtype=model.dtype))
    output_str = tokenizer.decode(output_tensors[0])
    return output_str

# 測試案例列表
test_cases = [
    {
        "question": "在法國大革命期間，為何拿破崙能成功地推翻法國共和政府並宣告自己為皇帝？",
        "answer": "拿破崙能成功地推翻法國共和政府並宣告自己為皇帝的原因包括：其軍事才能使他在混亂的法國大革命後期平定了內亂，贏得了人民的支持；他實行了獨裁政治，並廢除了選舉制度，使自己成為了無可挑戰的領導者；並且他利用了法國人民對於皇權的傳統崇拜，自封為法國皇帝，正式結束了法國共和政府的統治。"
    },
    {
        "question": "請解釋為何美國曾經在1947年實施馬歇爾計劃，以及該計劃對於歐洲後續統一有何影響？",
        "answer": "美國在1947年實施馬歇爾計劃，主要是為了遏制蘇聯的影響力，並防止歐洲國家轉向共產主義。此計劃對於歐洲後續的統一有重大影響，它透過經濟援助加強了歐洲各國的經濟合作與聯繫，進而形成歐洲共同體的基礎，最終導致歐洲聯盟的成立。"
    },
    {
        "question": "如果維京人的北歐探險在10世紀末期並未終止，他們是否有可能在哥倫布前發現美洲？",
        "answer": "否，維京人早在哥倫布前就已發現美洲。他們的探險活動在10世紀末期並未終止，他們在大約1000年左右就已經到達北美洲。"
    },
    {
        "question": "如果歐洲的文藝復興運動未曾發生，會對哪位科學家的理論產生最大的影響？",
        "answer": "哥白尼"
    },
    {
        "question": "如果美國獨立宣言並未在1776年簽署，那麼法國大革命的時間點可能會有所改變嗎？",
        "answer": "會"
    },
    {
        "question": "如果盧梭和菲利普二世在同一時期活著，那麼他們的對話可能會是關於什麼？",
        "answer": "他們的對話可能會是關於啟蒙時代的價值觀和統治理念，以及如何平衡人民的自由和國家的權威。此外，他們也可能會討論如何通過教育來提升國家的整體素質和實力。"
    },
    {
        "question": "法國大革命後的法國，和美國獨立戰爭後的美國，兩者的國家性質有何不同？",
        "answer": "法國大革命後的法國為民主共和國，重視公民的政治參與，而美國獨立戰爭後的美國為聯邦制國家，重視個人權利和自由。"
    },
    {
        "question": "如果克里斯托弗·哥倫布在1492年發現美洲並於1498年死亡，而瓦斯科·達伽馬在1497年發現了通往印度的海路，那麼在哥倫布死亡時，歐洲人是否已經知道到達印度的海路？",
        "answer": "是"
    },
    {
        "question": "如果美國獨立戰爭並未成功，那麼盧易斯安那購地是否還會發生？",
        "answer": "不會"
    },
    {
        "question": "如果光緒皇帝在1900年時仍然年幼，那麼誰是當時實際掌握清朝政權的人物？",
        "answer": "慈禧太后"
    },
    {
        "question": "如果工業革命發生在中國而非英國，會對全球歷史產生何種影響？",
        "answer": "如果工業革命發生在中國而非英國，中國可能會取代英國成為全球第一個工業化強國，進而影響全球政治和經濟的發展，並使中國的文化和思想在全球範圍內得到廣泛傳播。"
    },
    {
        "question": "如果明成祖朱棣與明太祖朱元璋的爭權奪位事件並未發生，那麼朱棣是否會發起靖難之役？",
        "answer": "不會"
    },
    {
        "question": "如果羅馬帝國在公元476年並未滅亡，那麼歐洲的中世紀是否會存在？",
        "answer": "不會"
    },
    {
        "question": "在中國古代，哪位皇帝在位時期，發生了兩次重要的歷史事件：一是漢武帝修建了長城，二是班超出使西域？",
        "answer": "秦始皇"
    },
    {
        "question": "如果哥倫布在1492年發現新大陸，而麥哲倫於1522年完成繞地球一周的航行，那麼哥倫布和麥哲倫是否有機會見面？",
        "answer": "不，哥倫布和麥哲倫沒有機會見面。"
    }
]

# 執行測試
print("==========================================")
print("     開始測試原始模型在多跳問題上的表現    ")
print("==========================================\n")

for idx, case in enumerate(tqdm(test_cases, desc="評估中")):
    question = case["question"]
    conversations = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": question}
    ]

    prompt = prompt_engine.get_prompt(conversations)
    output = _inference(tokenizer, model, generation_config, prompt)
    parsed = prompt_engine.parse_generated_str(output)

    print("-" * 80)
    print(f"問題 {idx+1}: {question}")
    print(f"模型回答：\n{parsed['content']}")
    print(f"✅ 正確答案：\n{case['answer']}")
    print("-" * 80)

print("==========================================")
print("           直接測試已完成。                ")
print("==========================================")


偵測到 Windows 環境，已調整信號處理以順利載入模型。
--------------------------------------------------
FlashAttention2 is not installed.


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.13s/it]


✅ 模型與分詞器載入成功！
     開始測試原始模型在多跳問題上的表現    



評估中:   7%|▋         | 1/15 [00:12<02:57, 12.69s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


--------------------------------------------------------------------------------
問題 1: 在法國大革命期間，為何拿破崙能成功地推翻法國共和政府並宣告自己為皇帝？
模型回答：
拿破崙成功推翻法國共和政府並宣告自己為皇帝的原因有幾點：

1.軍事才能：拿破崙是一位優秀的將領，擁有卓越的戰略和指揮能力。他在義大利戰爭中取得了勝利，並且迅速地征服了義大利大部分地區。這使得他獲得了聲望和權力。

2.政治手腕：拿破崙善於操縱輿論和利用媒體來傳播他的思想。他創立了一個強大的中央集權政府，並透過建立一個新的法律制度來鞏固自己的統治。

3.外交政策：拿破崙採取了一系列外交措施來加強法國的國際地位。他與其他歐洲國家簽訂了條約，併為法國提供了經濟援助。此外，他還積極參與國際事務，試圖影響歐洲的政治格局。

4.民族主義：拿破崙利用民族主義來團結人民，將他們對國家的愛國情懷轉化為支持他的力量。通過提倡法蘭西人的偉大和榮譽，他激發了公民的自豪感和忠誠度。

5.改革：拿破崙實施了一些重要的改革，如建立現代教育系統、發展交通基礎設施以及改進農業技術等。這些改革提高了人民的生活質量，使其更加依賴於拿破崙的統治。

總之，拿破崙能夠成功推翻法國共和政府並宣告自己為皇帝是由於他的軍事才能、政治手腕、外交政策、民族主義和改革努力所致。
✅ 正確答案：
拿破崙能成功地推翻法國共和政府並宣告自己為皇帝的原因包括：其軍事才能使他在混亂的法國大革命後期平定了內亂，贏得了人民的支持；他實行了獨裁政治，並廢除了選舉制度，使自己成為了無可挑戰的領導者；並且他利用了法國人民對於皇權的傳統崇拜，自封為法國皇帝，正式結束了法國共和政府的統治。
--------------------------------------------------------------------------------


...

--------------------------------------------------------------------------------
問題 13: 如果羅馬帝國在公元476年並未滅亡，那麼歐洲的中世紀是否會存在？
模型回答：
如果羅馬帝國在公元476年並未滅亡，那麼歐洲的中世紀可能會存在。這是因為羅馬帝國的崩潰標誌著一個重要的轉折點，導致了歐洲歷史上的一系列變化和發展。

當時，西羅馬帝國被日耳曼人入侵，並最終於476年滅亡。在此之後，各個地區開始獨立統治，形成了一系列小型國家或王國，這些國家隨後逐漸演變成中世紀的形式。

在沒有羅馬帝國的情況下，歐洲很有可能經歷了一段動盪不安、分裂且多樣化的時期。這種情況將促進地方勢力和貴族權力的興起，他們試圖建立自己的領土並對周圍地區施加影響力。

儘管如此，中世紀仍然是一個持續進行的過程，而不是一段固定的時間段。它通常被定義為從5至15世紀期間，歐洲社會和文化發生顯著變化的時期。因此，如果羅馬帝國在476年沒有滅亡，歐洲的中世紀仍然可能繼續存在，但其具體內容和特徵可能會有所不同。
✅ 正確答案：
不會
--------------------------------------------------------------------------------


評估中:  93%|█████████▎| 14/15 [01:51<00:05,  5.60s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


--------------------------------------------------------------------------------
問題 14: 在中國古代，哪位皇帝在位時期，發生了兩次重要的歷史事件：一是漢武帝修建了長城，二是班超出使西域？
模型回答：
漢武帝在位時期發生了兩次重要的歷史事件：一是修建長城，二是班超出使西域。
✅ 正確答案：
秦始皇
--------------------------------------------------------------------------------


評估中: 100%|██████████| 15/15 [01:55<00:00,  7.67s/it]

--------------------------------------------------------------------------------
問題 15: 如果哥倫布在1492年發現新大陸，而麥哲倫於1522年完成繞地球一周的航行，那麼哥倫布和麥哲倫是否有機會見面？
模型回答：
哥倫布在1492年發現新大陸，而麥哲倫於1522年完成繞地球一周的航行。兩人之間有機會見面，特別是在麥哲倫返回西班牙後。

當時，歐洲各國都對探索世界充滿熱情，因此可能會舉辦一些國際活動或聚會來討論這些成就。因此，哥倫布和麥哲倫很有可能在這些場合中相遇並互相交流他們的經歷與發現。
✅ 正確答案：
不，哥倫布和麥哲倫沒有機會見面。
--------------------------------------------------------------------------------
           直接測試已完成。                


In [3]:
prompt = prompt_engine.get_prompt(conversations)
output_str = _inference(tokenizer, model, generation_config, prompt)
result = prompt_engine.parse_generated_str(output_str)
print(result)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': '推理步驟：\n1. 愛因斯坦提出相對論。\n2. 愛因斯坦出生於德國的烏爾姆市。\n3. 德國是一個位於歐洲的國家，其首都是柏林。\n\n答案：提出相對論的科學家出生的國家的首都是柏林。'}


In [20]:
import os
import json
import torch
import sys
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForCausalLM

# 載入模型與 processor
model_name = "MediaTek-Research/Llama-Breeze2-3B-Instruct"

# 針對 Windows 系統移除信號超時檢查，以避免 AttributeError
# 此處我們檢查是否在非 Linux/macOS 的環境
if sys.platform == "win32":
    # 猴子補丁（Monkey Patch）來修復 Windows 不支援 signal.SIGALRM 的問題
    # 我們將其替換為一個無操作的函式，以便程式碼繼續執行
    import signal
    def no_op_handler(signum, frame):
        pass
    signal.signal = no_op_handler

print("偵測到 Windows 環境，已調整信號處理以順利載入模型。")
print("-" * 50)

try:
    # 使用 AutoProcessor 載入，它會處理 tokenizer 和 image processor 的所有細節
    processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
        trust_remote_code=True
    )
    model.eval() # 設定為評估模式
    print("✅ 模型與 Processor 載入成功！")
except Exception as e:
    print(f"❌ 載入模型時發生錯誤：{e}")
    sys.exit(1)

def get_answer_from_response(response_text):
    """
    從模型的生成文字中提取「最終答案」
    """
    response_parts = response_text.split("最終答案：")
    if len(response_parts) > 1:
        return response_parts[-1].strip()

    return response_text.strip()

# 20個多跳歷史問題，用於直接測試
test_cases = [
    {
        "question": "在法國大革命期間，為何拿破崙能成功地推翻法國共和政府並宣告自己為皇帝？",
        "answer": "拿破崙能成功地推翻法國共和政府並宣告自己為皇帝的原因包括：其軍事才能使他在混亂的法國大革命後期平定了內亂，贏得了人民的支持；他實行了獨裁政治，並廢除了選舉制度，使自己成為了無可挑戰的領導者；並且他利用了法國人民對於皇權的傳統崇拜，自封為法國皇帝，正式結束了法國共和政府的統治。"
    },
    {
        "question": "請解釋為何美國曾經在1947年實施馬歇爾計劃，以及該計劃對於歐洲後續統一有何影響？",
        "answer": "美國在1947年實施馬歇爾計劃，主要是為了遏制蘇聯的影響力，並防止歐洲國家轉向共產主義。此計劃對於歐洲後續的統一有重大影響，它透過經濟援助加強了歐洲各國的經濟合作與聯繫，進而形成歐洲共同體的基礎，最終導致歐洲聯盟的成立。"
    },
    {
        "question": "如果維京人的北歐探險在10世紀末期並未終止，他們是否有可能在哥倫布前發現美洲？",
        "answer": "否，維京人早在哥倫布前就已發現美洲。他們的探險活動在10世紀末期並未終止，他們在大約1000年左右就已經到達北美洲。"
    },
    {
        "question": "如果歐洲的文藝復興運動未曾發生，會對哪位科學家的理論產生最大的影響？",
        "answer": "哥白尼"
    },
    {
        "question": "如果美國獨立宣言並未在1776年簽署，那麼法國大革命的時間點可能會有所改變嗎？",
        "answer": "會"
    },
    {
        "question": "如果盧梭和菲利普二世在同一時期活著，那麼他們的對話可能會是關於什麼？",
        "answer": "他們的對話可能會是關於啟蒙時代的價值觀和統治理念，以及如何平衡人民的自由和國家的權威。此外，他們也可能會討論如何通過教育來提升國家的整體素質和實力。"
    },
    {
        "question": "法國大革命後的法國，和美國獨立戰爭後的美國，兩者的國家性質有何不同？",
        "answer": "法國大革命後的法國為民主共和國，重視公民的政治參與，而美國獨立戰爭後的美國為聯邦制國家，重視個人權利和自由。"
    },
    {
        "question": "如果克里斯托弗·哥倫布在1492年發現美洲並於1498年死亡，而瓦斯科·達伽馬在1497年發現了通往印度的海路，那麼在哥倫布死亡時，歐洲人是否已經知道到達印度的海路？",
        "answer": "是"
    },
    {
        "question": "如果美國獨立戰爭並未成功，那麼盧易斯安那購地是否還會發生？",
        "answer": "不會"
    },
    {
        "question": "如果光緒皇帝在1900年時仍然年幼，那麼誰是當時實際掌握清朝政權的人物？",
        "answer": "慈禧太后"
    },
    {
        "question": "如果工業革命發生在中國而非英國，會對全球歷史產生何種影響？",
        "answer": "如果工業革命發生在中國而非英國，中國可能會取代英國成為全球第一個工業化強國，進而影響全球政治和經濟的發展，並使中國的文化和思想在全球範圍內得到廣泛傳播。"
    },
    {
        "question": "如果明成祖朱棣與明太祖朱元璋的爭權奪位事件並未發生，那麼朱棣是否會發起靖難之役？",
        "answer": "不會"
    },
    {
        "question": "如果羅馬帝國在公元476年並未滅亡，那麼歐洲的中世紀是否會存在？",
        "answer": "不會"
    },
    {
        "question": "在中國古代，哪位皇帝在位時期，發生了兩次重要的歷史事件：一是漢武帝修建了長城，二是班超出使西域？",
        "answer": "秦始皇"
    },
    {
        "question": "如果哥倫布在1492年發現新大陸，而麥哲倫於1522年完成繞地球一周的航行，那麼哥倫布和麥哲倫是否有機會見面？",
        "answer": "不，哥倫布和麥哲倫沒有機會見面。"
    }
]

# 測試原始模型
def test_original_model_direct(model, processor, test_cases, max_gen_tokens=150):
    print("==========================================")
    print("  開始直接測試原始模型在多跳問題上的表現  ")
    print("==========================================\n")

    for i, case in enumerate(tqdm(test_cases, desc="評估中")):
        question = case["question"]
        prompt = f"問題：{question}\n請逐步推理並回答："
        
        # 修正：使用 apply_chat_template 函數來準備模型輸入，並直接使用 tokenizer
        messages = [{"role": "user", "content": prompt}]
        full_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # 使用 tokenizer 進行編碼，然後手動添加 pixel_values=None
        inputs = processor.tokenizer(
            full_prompt,
            return_tensors="pt"
        ).to(model.device)
        inputs['pixel_values'] = None

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_gen_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=processor.eos_token_id
            )
        
        response = processor.decode(outputs[0], skip_special_tokens=True)
        
        # 提取模型回答，並移除 chat template 部分
        full_prompt_template = processor.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=False,
            add_generation_prompt=True
        )
        response_text = response.split(full_prompt_template.strip())[-1].strip()

        pred_answer = get_answer_from_response(response_text)
        ref_answer = case["answer"].strip()
        is_correct = "✅ 正確" if pred_answer == ref_answer else "❌ 錯誤"

        print("-" * 50)
        print(f"問題 {i+1}: {question}\n")
        print(f"✅ 正確答案: {ref_answer}")
        print(f"🤖 模型回答: {pred_answer}")
        print(f"🎯 判斷結果: {is_correct}\n")
        print(f"--- 完整模型回應 --- \n{response_text}\n-----------------------\n")
    
    print("==========================================")
    print("          直接測試已完成。                ")
    print("==========================================")

if __name__ == "__main__":
    test_original_model_direct(model, processor, test_cases)


偵測到 Windows 環境，已調整信號處理以順利載入模型。
--------------------------------------------------


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.52s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


✅ 模型與 Processor 載入成功！
  開始直接測試原始模型在多跳問題上的表現  



評估中:   0%|          | 0/15 [00:00<?, ?it/s]


AttributeError: PreTrainedTokenizerFast has no attribute tokenizer

In [24]:
print("最終答案：不會")

最終答案：不會
